The first cell must be executed but not edited. Next cells are numerical experiments and can be edited to change parameters and graphics.

In [1]:
%matplotlib inline
import time
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import randn
from __future__ import print_function # for Python 2/3 compatibility
from matplotlib import animation, rc
from IPython.display import display, HTML

# module Y.X corresponds to file Y/X.py

from scipy import sparse as sparse
import dynamics.time_step as time_step

def col_vec(data):
    vec=np.zeros((data.size,1))
    vec[:,0]=data
    return vec
def row_vec(data):
    vec=np.zeros((1,data.size))
    vec[0,:]=data
    return vec


####################### Horizontal mesh #########################

class Periodic_FD:
    # staggering
    # m0  m1  m2  m3
    #   u0  u1  u2  u3
    def __init__(self, N):
        self.N=N
        ones=np.ones(N);

        right = sparse.spdiags([ones],[0],N,N, format='csr')
        left = sparse.spdiags([ones],[-1],N,N, format='csr')
        left[0,-1]=1
        di = right-left
        mi = .5*(right+left)
        self.di=lambda f:di.dot(f)
        self.mi=lambda f:mi.dot(f)
        self.xi = col_vec(np.linspace(0,N-1,N))-N/2.
        self.onesi = np.ones((N,1))
        self.zeroi = np.zeros((N,1))
       
        right = sparse.spdiags([ones],[1],N,N, format='csr')
        right[-1,0]=1
        left = sparse.spdiags([ones],[0],N,N, format='csr')
        dj = right-left
        mj = .5*(right+left)
        self.dj=lambda f:dj.dot(f)
        self.mj=lambda f:mj.dot(f)
        self.xj = col_vec(np.linspace(0.5,N-.5,N))-N/2.
        self.onesj = np.ones((N,1))
        
        self.onesk = np.ones((1,N))
        self.onesl = np.ones((1,N))
        self.xk = row_vec(np.linspace(0,N-1,N))-N/2.
        self.xl = row_vec(np.linspace(0.5,N-.5,N))-N/2.


    def hstack(self,fields):
        return np.array(np.hstack(fields))
    def up(self,f):
        N=self.N
        return self.hstack( (f[:,N-1:N] , f[:,:-1] ) )
    def down(self,f):
        N=self.N
        return self.hstack( ( f[:,1:] , f[:,0:1] ) )
    def dk(self, fl):
        return fl-self.up(fl)
    def mk(self, fl):
        return .5*(fl+self.up(fl))
    def dl(self, fk):
        return self.down(fk)-fk
    def ml(self, fk):
        return .5*(fk+self.down(fk))


############################ x-y rectangular mesh ##########################

class Rectangle:
    def __init__(self,xmesh,ymesh):
        self.xmesh, self.ymesh = xmesh, ymesh
        self.Nx, self.Ny = xmesh.N, ymesh.N
        self.di, self.dj, self.mi, self.mj = xmesh.di, xmesh.dj, xmesh.mi, xmesh.mj
        self.dk, self.dl, self.mk, self.ml = ymesh.dk, ymesh.dl, xmesh.mk, xmesh.ml
        
        self.xik = xmesh.xi*ymesh.onesk
        self.xjk = xmesh.xj*ymesh.onesk
        self.xil = xmesh.xi*ymesh.onesl
        self.xjl = xmesh.xj*ymesh.onesl
        
        self.yik = xmesh.onesi*ymesh.xk
        self.yil = xmesh.onesi*ymesh.xl
        self.yjk = xmesh.onesj*ymesh.xk
        self.yjl = xmesh.onesj*ymesh.xl
    # double averaging used for Coriolis
    def mjk(self, data):
        return self.mj(self.mk(data))
    def mil(self, data):
        return self.mi(self.ml(data))
    def mjl(self, data):
        return self.mj(self.ml(data))

############################# Grid staggering ###########################

# Due to grid staggering, variable names for fields have a symbolic suffix indicating where the field is known
# In the x direction symbols i and j are used ; i is for cells and j for boundaries between cell
# In the y direction symblos k and l are used ; k is for cells and l for boundaries between cells
# A finite difference operator taking input data on grid i (resp. j,k,l) computes an output on grid j (resp. i,l,k)
# The name of an operator has a suffix indicating on which grid the RESULT is

############################## Wave equation ############################

# convention : eta is the relative deviation of h from its reference value H
# i.e. h'=H.eta and the free surface is at H(1+eta)
# the parameters are g'H and f
 
class Wave:
    def __init__(self, mesh, dx, dy, f, gH=1.):
        self.mesh = mesh
        self.dx,self.dy = dx,dy
        self.gH, self.f = gH, f
    def curl_jl(self, u_jk, v_il):
        dx,dy,mesh = self.dx, self.dy, self.mesh
        di,dj,dk,dl = mesh.di, mesh.dj, mesh.dk, mesh.dl
        zeta_jl = dj(v_il)/dx - dl(u_jk)/dy
        return zeta_jl
    def bwd_fast_slow(self, flow, tau):
        h_ik, u_jk, v_il = flow
        dflow = self.equation(*flow)
        return ( flow, (0.,0.,0.), dflow )
    def equation(self, eta_ik, u_jk, v_il):
        dx,dy,mesh,f,gH = self.dx, self.dy, self.mesh, self.f, self.gH
        di,dj,dk,dl,mjk,mil = mesh.di, mesh.dj, mesh.dk, mesh.dl, mesh.mjk, mesh.mil
        # dh/dt + du/dx + dv/dy = 0
        deta_ik = - di(u_jk)/dx-dk(v_il)/dy
        # du/dt - fv + gH.dh/dx = 0
        du_jk =  f*mesh.mjk(v_il) - (gH/dx)*dj(eta_ik)
        # dv/dt + fu + gH.dh/dy = 0
        dv_il = -f*mesh.mil(u_jk) - (gH/dy)*dl(eta_ik) 
        return deta_ik, du_jk, dv_il

############################## Shallow-water equations ############################

# convention : eta is now g'h
# the only parameter is f

# Enstrophy-conserving scheme from Sadourny (1975)
# "The Dynamics of Finite-Difference Models of Shallow-Water Equations"

class SaintVenant(Wave):
    def equation(self, gh_ik, u_jk, v_il):
        dx,dy,mesh,f,gH = self.dx, self.dy, self.mesh, self.f, self.gH
        di,dj,dk,dl = mesh.di, mesh.dj, mesh.dk, mesh.dl   # finite-difference operators
        mi,mj,mk,ml = mesh.mi, mesh.mj, mesh.mk, mesh.ml   # averaging operators
        # dh/dt + d/dx(hu) + d/dy(hv) = 0
        U_jk = mj(gh_ik)*u_jk
        V_il = ml(gh_ik)*v_il
        dgh_ik = - di(U_jk)/dx-dk(V_il)/dy
        # B = gH + (u^2+v^2)/2
        B_ik = mi(u_jk*u_jk) + mk(v_il*v_il)
        B_ik = gh_ik + .5*B_ik
        # zeta = dv/dx-du/dy, q = (zeta+f)/gh
        zeta_jl = dj(v_il)/dx-dl(u_jk)/dy
        q_jl = (zeta_jl+f)/mesh.mjl(gh_ik)
        q_il, q_jk = mi(q_jl), mk(q_jl)
        # du/dt - qV + dB/dx = 0
        du_jk =  q_jk*mesh.mjk(V_il) - dj(B_ik)/dx
        # dv/dt + qU + dB/dy = 0
        dv_il = -q_il*mesh.mil(U_jk) - dl(B_ik)/dy 
        return dgh_ik, du_jk, dv_il

##################### Run the model and create animation ##########################

# see http://tiao.io/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

class TimeSequence:
    def __init__(self, mesh, model, scheme, Nstep, flow0, ax):
        self.mesh, self.model, self.scheme =  mesh, model, scheme
        self.Nstep, self.flow0, self.ax = Nstep, flow0, ax
    def draw(self, flow, i):
        self.flow = flow
        mesh, scheme, ax = self.mesh, self.scheme, self.ax
        dx, dy = self.model.dx, self.model.dy
        eta_ik,u_jk,v_il = flow
        zeta_jl = mesh.dj(v_il)/dx - mesh.dl(u_jk)/dy
        ax.clear()
        my_figure(ax, dx*mesh.xik,dy*mesh.yik, i*Texp,
                  eta_ik, mesh.mi(u_jk), mesh.mk(v_il),
                  mesh.mi(mesh.mk(zeta_jl)) )
    def start(self):
        self.flow = self.flow0
        self.time = time.time()
        return (self.ax,) # this function must return a tuple, do not remove comma
#        return self.draw(self.flow0, 0)
    def animate(self,i):
        scheme, flow = self.scheme, self.flow
        self.draw(flow, i)
        for ii in range(self.Nstep):
            flow = scheme.next(flow)
        self.flow = flow
        dtime = time.time()-self.time
        if dtime>3.:
            print('Frame %d/%d'%(i,Nexp))
            self.time = self.time + dtime
        return (self.ax,)

def experiment(Model, Nx=128, Ny=128, courant=0.5, fps=5, filename=None, ):
    dx, dy = Lx/Nx, Ly/Ny
    dt = courant*dx/np.sqrt(gH)

    xmesh = Periodic_FD(Nx)
    ymesh = Periodic_FD(Ny)
    mesh = Rectangle(xmesh,ymesh)

    model = Model(mesh,dx,dy,f,gH)
    Nstep = int((1.*Texp)/dt+1.)
    dt = (1.*Texp)/Nstep
    scheme = time_step.ARK2(model.bwd_fast_slow, dt)

    eta0_ik, junk, junk = initial_condition(dx*mesh.xik,dy*mesh.yik)
    junk, u0_jk, junk   = initial_condition(dx*mesh.xjk,dy*mesh.yjk)
    junk, junk, v0_il   = initial_condition(dx*mesh.xil,dy*mesh.yil)
    flow = (eta0_ik,u0_jk,v0_il)

    if filename is None:
        for j in range(Nexp):
            for i in range(Nstep): flow = scheme.next(flow)
            eta_ik, u_jk, v_il = flow
            zeta_jl = mesh.dj(v_il)/dx - mesh.dl(u_jk)/dy
            plt.figure(figsize=(5,5))
            my_figure(plt, mesh.xik*dx, mesh.yik*dy, j*Texp,
                      eta_ik, mesh.mi(u_jk), mesh.mk(v_il),
                      mesh.mi(mesh.mk(zeta_jl)) )
            plt.show()
            plt.close()
    else:
        fig=plt.figure(figsize=(5,5))
        seq = TimeSequence(mesh, model, scheme, Nstep, flow, plt.gca())
        anim = animation.FuncAnimation(fig,seq.animate, Nexp, 
                                       init_func=seq.start,
                                       interval=1e3/fps,blit=False )
        anim.save(filename, codec='h264')

def show_movie(filename, width=400):
    return HTML('<video width=%d controls><source src="%s" type="video/mp4"></video>'%(width,filename) )

First experiment : one-dimensional geostrophic adjustment in the linearized shallow-water equations.

Parameters are $f$ and $gH=g'H$. Try $f=0$ (non-rotating) and $f=1$ (rotating). 

Make sure to clear output (in the menu : Cell -> Clear Output) before re-running the cell otherwise the movie may not be udpated.

In [0]:
f=1.            # Coriolis parameter
Nexp=100        # number of frames in movie
Texp=1.         # time interval between two movie frames
xlim=(-10., 10) # show only this part of the domain

#-------------------------------------------------------------------------------

Lx,Ly,gH = 200.,20., 1. # Domain size, g'*H

def initial_condition(x,y):
    eta = np.exp(-.5*x**2)
    u = 0*x # make sure u is an array like x
    v = 0*x
    return eta, u, v

def my_figure(ax, x,y,t, eta, u, v, zeta):
    def plot(var, name): ax.plot(x,var[:,0],label=name)
    x = x[:,0]
    plot(eta,'eta')
    plot(u,'u')
    plot(v,'v')
    plt.xlim(xlim)
    plt.ylim((-0.4,1.0))
    plt.legend()
    plt.title('t=%f'%t)
    
#experiment(Wave, Ny=2, Nx=2048)
experiment(Wave, Ny=2, Nx=2048, fps=10, filename='linear1D.mp4')
show_movie('linear1D.mp4', width=600)

# Remark : make sure to clear output (Cell -> Clear Output) before re-running this cell otherwise the movie may not be refreshed

Second experiment : two-dimensional geostrophic adjustment in the linearized shallow-water equations.

Parameters are $f$ and $gH=g'H$ 

Try $f=0$ (non-rotating) and $f=1$ (rotating). You can choose to show various quantities (vorticity, kinetic energy, ...)

Make sure to clear output (in the menu : Cell -> Clear Output) before re-running the cell otherwise the movie may not be udpated.

In [0]:
f=1.         # Coriolis parameter
Nexp=100     # number of frames in movie
Texp=.5      # time interval between two movie frames
varname='q'  # quantity shown in the plot : choose among h, zeta, q, ke, u, v

#-------------------------------------------------------------------------------

Lx,Ly,gH = 20.,20., 1. # Domain size, Coriolis, g'*H

def initial_condition(x,y):
    eta = np.exp(-.5*(x**2+y**2))
    u = 0*x # make sure u is an array like x
    v = 0*x
    return eta, u, v

def my_figure(ax, x,y,t, eta, u, v, zeta):
    var = {'h':eta, 'zeta':zeta, 'q':zeta-eta, 
           'ke':.5*(u*u+v*v), 'u':u, 'v':v}[varname]
    ax.contourf(x,y,var,20)
    plt.title('%s at t=%f'%(varname,t))

experiment(Wave, filename='linear2D.mp4', fps=10)
show_movie('linear2D.mp4', width=500)

Third experiment : two-dimensional geostrophic adjustment and slow dynamics in the shallow-water equations.
Two initial conditions are proposed : one (steep_vortex) is an isolated 'bump' and the other one (modon) is made of two bumps of opposite sign near each other.

Parameter : $f$ 

Try $f=0$ (non-rotating) and $f=1$ (rotating). You can choose to show various quantities (vorticity, kinetic energy, ...). If you feel that nothing happens, increase Texp to make a longer experiment. Interesting simulations may take a several minutes to compute.

Make sure to clear output (in the menu : Cell -> Clear Output) before re-running the cell otherwise the movie may not be udpated.

In [3]:
f=1.          # Coriolis parameter
Nexp=100      # number of frames in movie
Texp=20        # time interval between two movie frames
varname='zeta'  # quantity shown in the plot : choose among 'gh', 'gh-gh0', 'zeta', 'pv', 'ke', 'u', 'v'

initial_condition='modon' # 'modon' or 'steep_vortex'

#----------------------------------------------------------------

Lx,Ly,gH = 10.,10.,1. # Domain size

def modon(x,y):
    gh = 1.+.2*np.exp(-.5*((x-.5)**2+y**2))-.2*np.exp(-.5*((x+.5)**2+y**2))
    u = 0*x # make sure u is an array like x
    v = 0*x
    return gh, u, v

def steep_vortex(x,y):
    r2 = x**2+y**2
    F = np.exp(-.5*r2**2)
    dF = -r2*F # dF/d(r^2)
    amplitude = 0.2
    gh = 1.+amplitude*F  + .001*randn(*x.shape)
    if f>0: # impose geostrophic balance initially
        Fy = 2.*y *dF # dF/dy = dr^2/dy * dF/dr^2
        Fx = 2.*x *dF
        u , v = -amplitude*Fy/f, amplitude*Fx/f 
    else:
        u = 0*x # make sure u is an array like x
        v = 0*x
    return gh, u, v

def my_figure(ax, x,y,t, gh, u, v, zeta):
    gh0, u0, v0 = initial_condition(x,y)
    var = {'gh':gh, 'gh-gh0':gh-gh0, 'zeta':zeta, 'pv':(f+zeta)/gh,
           'ke':.5*(u*u+v*v), 'u':u, 'v':v}[varname]
    ax.contourf(x,y,var,20)
    plt.title('%s at t=%f'%(varname,t))
    print('t=%f, vmax=%f'%(t,u.max()), var.min(), var.max())

initial_condition = {'modon':modon, 'steep_vortex':steep_vortex}[initial_condition]

experiment(SaintVenant, Nx=64, Ny=64, filename='SaintVenant.mp4')
show_movie('SaintVenant.mp4')

t=0.000000, vmax=0.000000 0.0 0.0


t=20.000000, vmax=0.027466 -0.0724215756029 0.0893284657031


t=40.000000, vmax=0.026373 -0.0520088362636 0.0589063399877


t=60.000000, vmax=0.047317 -0.0900291299278 0.110201335044


Frame 3/100
t=80.000000, vmax=0.023678 -0.0484880060139 0.0567155631582


...

t=1900.000000, vmax=0.032902 -0.0814394460405 0.095613098614


t=1920.000000, vmax=0.020558 -0.0751669392243 0.0791075317955


Frame 96/100
t=1940.000000, vmax=0.031691 -0.0788923484568 0.091026636076


t=1960.000000, vmax=0.016918 -0.0729524959061 0.0786276150346


t=1980.000000, vmax=0.028419 -0.0845095093075 0.0931844244845
